# Experiment: Stateful Time-Resolved Thermal Simulation

This notebook runs the simulation experiment directly and visualizes the results.

In [ ]:
import sys
import os
from pathlib import Path

# Ensure we can import from src and experiments
sys.path.append(str(Path("..").resolve()))

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import Dropdown, IntSlider, interact
from matplotlib import animation

from experiments.stateful_time_resolved import run_experiment
from neural_pbf.schemas.state import ThermalStates

In [ ]:
# Run the experiment
# This will return the final states object directly
print("Running experiment...")
states = run_experiment(tracking_enabled=True, artifact_enabled=True, diagnostics_enabled=True)
print("Experiment complete!")

snap = states.snapshots
print("Meta:", states.meta)

## Visualization

In [ ]:
def show_frame(field: str, i: int, vmin=None, vmax=None):
    arr = getattr(snap, field)[i, 0].numpy()  # [H,W]
    t = float(snap.t[i].item())
    ev = int(snap.event_idx[i].item())

    plt.figure(figsize=(5, 4))
    plt.imshow(arr, origin="lower", vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.title(f"{field} | frame={i} | t={t:.3f} | event={ev}")
    plt.tight_layout()
    plt.show()

In [ ]:
# Precompute ranges
T_all = snap.T[:, 0].numpy()
T_vmin, T_vmax = float(T_all.min()), float(T_all.max())

E_all = snap.E_acc[:, 0].numpy()
E_vmin, E_vmax = float(E_all.min()), float(E_all.max())

S_all = snap.t_since[:, 0].numpy()
S_vmin, S_vmax = float(S_all.min()), float(S_all.max())

R_all = snap.cooling_rate[:, 0].numpy()
# Handle NaNs in cooling rate for min/max
R_all_clean = R_all[~np.isnan(R_all)]
if R_all_clean.size > 0:
    R_min, R_max = float(R_all_clean.min()), float(R_all_clean.max())
else:
    R_min, R_max = 0.0, 1.0

ranges = {
    "T": (T_vmin, T_vmax),
    "E_acc": (E_vmin, E_vmax),
    "t_since": (S_vmin, S_vmax),
    "cooling_rate": (R_min, R_max),
}

def view(i=0, field="T"):
    vmin, vmax = ranges[field]
    show_frame(field, i, vmin=vmin, vmax=vmax)

interact(
    view,
    i=IntSlider(min=0, max=snap.T.shape[0] - 1, step=1, value=0),
    field=Dropdown(options=["T", "E_acc", "t_since", "cooling_rate"], value="T"),
);

In [ ]:
# Animation (T)
field = "T"
data = getattr(snap, field)[:, 0].numpy()
times = snap.t.numpy()
events = snap.event_idx.numpy()
vmin, vmax = float(data.min()), float(data.max())

fig, ax = plt.subplots(figsize=(5, 4))
im = ax.imshow(data[0], origin="lower", vmin=vmin, vmax=vmax)
plt.colorbar(im, ax=ax)
title = ax.set_title("")

def update(frame):
    im.set_array(data[frame])
    title.set_text(f"{field} | frame={frame} | t={times[frame]:.3f} | event={events[frame]}")
    return (im, title)

ani = animation.FuncAnimation(fig, update, frames=data.shape[0], interval=150, blit=False)
plt.close(fig)
ani